In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s5e9/sample_submission.csv
/kaggle/input/playground-series-s5e9/train.csv
/kaggle/input/playground-series-s5e9/test.csv


In [2]:
data = pd.read_csv("/kaggle/input/playground-series-s5e9/train.csv")
data.head(10)

,id,RhythmScore,AudioLoudness,VocalContent,AcousticQuality,InstrumentalScore,LivePerformanceLikelihood,MoodScore,TrackDurationMs,Energy,BeatsPerMinute
0,0,0.603610,-7.636942,0.023500,0.000005,0.000001,0.051385,0.409866,290715.6450,0.826267,147.53020
1,1,0.639451,-16.267598,0.071520,0.444929,0.349414,0.170522,0.651010,164519.5174,0.145400,136.15963
2,2,0.514538,-15.953575,0.110715,0.173699,0.453814,0.029576,0.423865,174495.5667,0.624667,55.31989
3,3,0.734463,-1.357000,0.052965,0.001651,0.159717,0.086366,0.278745,225567.4651,0.487467,147.91212
4,4,0.532968,-13.056437,0.023500,0.068687,0.000001,0.331345,0.477769,213960.6789,0.947333,89.58511
5,5,0.594579,-11.715431,0.101701,0.000005,0.259634,0.024300,0.639254,161012.2221,0.938333,151.43602
6,6,0.714958,-8.809725,0.023500,0.471139,0.000001,0.356775,0.724783,243432.5794,0.336800,151.63439
7,7,0.703726,-10.482610,0.037751,0.115727,0.000001,0.024300,0.773899,293848.8919,0.722000,123.31545
8,8,0.455598,-1.357000,0.162537,0.173626,0.000001,0.288465,0.651137,271215.8237,0.947267,133.59806
9,9,0.431498,-5.935425,0.023500,0.439565,0.184462,0.114921,0.025600,222030.4794,0.950867,117.39790


In [3]:
data.describe()

,id,RhythmScore,AudioLoudness,VocalContent,AcousticQuality,InstrumentalScore,LivePerformanceLikelihood,MoodScore,TrackDurationMs,Energy,BeatsPerMinute
count,524164.000000,524164.000000,524164.000000,524164.000000,524164.000000,524164.000000,524164.000000,524164.000000,524164.000000,524164.000000,524164.000000
mean,262081.500000,0.632843,-8.379014,0.074443,0.262913,0.117690,0.178398,0.555843,241903.692949,0.500923,119.034899
std,151313.257586,0.156899,4.616221,0.049939,0.223120,0.131845,0.118186,0.225480,59326.601501,0.289952,26.468077
min,0.000000,0.076900,-27.509725,0.023500,0.000005,0.000001,0.024300,0.025600,63973.000000,0.000067,46.718000
25%,131040.750000,0.515850,-11.551933,0.023500,0.069413,0.000001,0.077637,0.403921,207099.876625,0.254933,101.070410
50%,262081.500000,0.634686,-8.252499,0.066425,0.242502,0.074247,0.166327,0.564817,243684.058150,0.511800,118.747660
75%,393122.250000,0.739179,-4.912298,0.107343,0.396957,0.204065,0.268946,0.716633,281851.658500,0.746000,136.686590
max,524163.000000,0.975000,-1.357000,0.256401,0.995000,0.869258,0.599924,0.978000,464723.228100,1.000000,206.037000


In [4]:
output = pd.DataFrame(columns = ["id", "BeatsPerMinute"])

In [5]:
from tensorflow.keras import layers
from tensorflow import keras

2025-09-16 04:40:54.783135: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1757997655.028275      13 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1757997655.106628      13 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [6]:
X = data.drop(columns = ['id', 'BeatsPerMinute'])
Y = data['BeatsPerMinute']
data.shape

(524164, 11)

In [7]:
model = keras.Sequential([
    layers.Dense(256, activation='relu', input_shape=(X.shape[1],)),
    layers.BatchNormalization(),
    layers.Dropout(0.3),
    layers.Dense(256, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.3),
    layers.Dense(1, activation = 'linear')
])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
2025-09-16 04:41:13.255634: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


In [8]:
model.compile(
    optimizer = 'adam',
    loss = 'mean_squared_error',
    metrics = ['mae']
)

In [9]:
from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(
    monitor = 'val_loss',
    patience = 10,
    restore_best_weights = True,
)

In [10]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X, Y, test_size=0.2, random_state=42
)

In [11]:
model.fit(
    X_train, y_train,
    validation_data = (X_test, y_test),
    epochs = 5,
    callbacks = [early_stopping],
    verbose =1
)

Epoch 1/5
13105/13105 ━━━━━━━━━━━━━━━━━━━━ 65s 5ms/step - loss: 2308.8364 - mae: 33.8941 - val_loss: 703.3703 - val_mae: 21.2489
Epoch 2/5
13105/13105 ━━━━━━━━━━━━━━━━━━━━ 64s 5ms/step - loss: 733.5536 - mae: 21.6806 - val_loss: 700.8192 - val_mae: 21.2012
Epoch 3/5
13105/13105 ━━━━━━━━━━━━━━━━━━━━ 70s 5ms/step - loss: 724.8834 - mae: 21.5564 - val_loss: 699.8874 - val_mae: 21.1915
Epoch 4/5
13105/13105 ━━━━━━━━━━━━━━━━━━━━ 69s 5ms/step - loss: 726.5192 - mae: 21.5775 - val_loss: 701.8829 - val_mae: 21.2235
Epoch 5/5
13105/13105 ━━━━━━━━━━━━━━━━━━━━ 66s 5ms/step - loss: 724.4639 - mae: 21.5463 - val_loss: 700.2852 - val_mae: 21.1945


In [12]:
test_data = pd.read_csv('/kaggle/input/playground-series-s5e9/test.csv')

In [13]:
test_data_new = test_data.drop(columns = ['id'])

In [14]:
output['id'] = test_data['id']
output['BeatsPerMinute'] = 0

In [15]:
pred = model.predict(test_data_new)
pred[:10]

5461/5461 ━━━━━━━━━━━━━━━━━━━━ 10s 2ms/step


array([[120.07297 ],
       [118.4323  ],
       [120.79655 ],
       [119.66139 ],
       [119.73984 ],
       [120.239395],
       [118.28705 ],
       [117.59436 ],
       [119.37173 ],
       [117.368576]], dtype=float32)

In [16]:
output['BeatsPerMinute'] = pred

In [17]:
output.to_csv("/kaggle/working/submission.csv", index = False)

In [18]:
# import shutil
# import os
# # Define source and destination
# src = "/kaggle/working/submission.csv"
# dst = "/kaggle/working/output_submission.csv"
# # Check if submission file exists
# if os.path.exists(src):
#    shutil.copy(src, dst)
#    print(f"✅ Submission file saved as: {dst}")
# else:
#    print("⚠️ submission.csv not found. Make sure it is generated before this cell runs.")